### Load tensorflow

In [1]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read the data
<font size="2">Data for this exercise can be downloaded from http://www.manythings.org/anki/</font>

In [0]:
#You can use wget to download the file directly
!wget http://www.manythings.org/anki/hin-eng.zip --quiet

In [0]:
import zipfile
import io

#Read the zip file
zf = zipfile.ZipFile('hin-eng.zip', 'r')

#Extract data from zip file
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [6]:
len(data)

401994

In [8]:
data[0:120]

'Wow!\tवाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #6179147 (fastrizwaan)\nHelp!\tबचाओ!\tCC-BY 2.0 (Fra'


### Extract Source and Target Language pairs

In [12]:
#Split by newline character
data =  data.split('\n')

#Show some Data
data[100:105]

["I don't know.\tमुझे नहीं पता।\tCC-BY 2.0 (France) Attribution: tatoeba.org #349064 (fatih) & #609376 (minshirui)",
 "I don't know.\tमुझे नहीं मालूम।\tCC-BY 2.0 (France) Attribution: tatoeba.org #349064 (fatih) & #609377 (minshirui)",
 'I have a car.\tमेरे पास एक गाड़ी है।\tCC-BY 2.0 (France) Attribution: tatoeba.org #252272 (CK) & #477720 (minshirui)',
 'I have a dog.\tमेरे पास एक कुत्ता है।\tCC-BY 2.0 (France) Attribution: tatoeba.org #378502 (CK) & #443037 (minshirui)',
 'I understand.\tमैं समझता हूँ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #433468 (CK) & #588495 (minshirui)']

In [0]:
temp = "I don't know.\tमुझे नहीं पता।\tCC-BY 2.0 (France) Attribution: tatoeba.org #349064 (fatih) & #609376 (minshirui)"

In [16]:
temp.split("\t")

["I don't know.",
 'मुझे नहीं पता।',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #349064 (fatih) & #609376 (minshirui)']

In [0]:
len(data)

2786

### Separate Source and Target pairs

In [0]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list

#Iterate over data
for line in data:
    try:
        in_txt, out_txt, attr = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error        

### Separate Source and Target pairs..

In [18]:
encoder_text[100:105]

["I don't know.",
 "I don't know.",
 'I have a car.',
 'I have a dog.',
 'I understand.']

In [19]:
decoder_text[100:105]

['<start> मुझे नहीं पता। <end>',
 '<start> मुझे नहीं मालूम। <end>',
 '<start> मेरे पास एक गाड़ी है। <end>',
 '<start> मेरे पास एक कुत्ता है। <end>',
 '<start> मैं समझता हूँ। <end>']

### Tokenize Source language sentences

In [22]:
encoder_text[100:105]

["I don't know.",
 "I don't know.",
 'I have a car.',
 'I have a dog.',
 'I understand.']

In [23]:
#Tokenizer for source language
encoder_t = tf.keras.preprocessing.text.Tokenizer()
encoder_t.fit_on_texts(encoder_text) #Fit it on Source sentences
encoder_seq = encoder_t.texts_to_sequences(encoder_text) #Convert sentences to numbers 
encoder_seq[100:105] #Display some converted sentences

[[2, 27, 43], [2, 27, 43], [2, 14, 6, 97], [2, 14, 6, 124], [2, 209]]

In [0]:
#encoder_t.word_index

In [26]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

Maximum sentence length for Source language:  22


In [27]:
#Source language Vocablury
encoder_vocab_size = len(encoder_t.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

Source language vocablury size:  2376


### Tokenize Target language sentences

In [0]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
decoder_t = tf.keras.preprocessing.text.Tokenizer()
decoder_t.fit_on_texts(decoder_text) #Fit it on target sentences
decoder_seq = decoder_t.texts_to_sequences(decoder_text) #Convert sentences to numbers 

In [29]:
decoder_text[100:105]

['<start> मुझे नहीं पता। <end>',
 '<start> मुझे नहीं मालूम। <end>',
 '<start> मेरे पास एक गाड़ी है। <end>',
 '<start> मेरे पास एक कुत्ता है। <end>',
 '<start> मैं समझता हूँ। <end>']

In [30]:
decoder_seq[100:105] #Display some converted sentences

[[1, 10, 5, 615, 2],
 [1, 10, 5, 1479, 2],
 [1, 28, 40, 21, 101, 3, 2],
 [1, 28, 40, 21, 202, 3, 2],
 [1, 6, 764, 22, 2]]

In [31]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_t.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

Maximum sentence length for Target language:  27
Target language vocablury size:  2974


### Compare different sentences length

In [32]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[100]))
print('Length for sentence number 2000: ', len(encoder_seq[2000]))

Length for sentence number 100:  3
Length for sentence number 2000:  6


In [33]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 2000: ', len(decoder_seq[2000]))

Length for sentence number 100:  5
Length for sentence number 2000:  9


### How do we make it same?

### Padding the sentences

In [0]:
#Source sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq, 
                                                                   maxlen=max_encoder_seq_length, #22
                                                                   padding='pre')

#Target Sentences
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq, 
                                                                   maxlen=max_decoder_seq_length, #27
                                                                   padding='post')

In [37]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

Source data shape:  (2785, 22)
Target data shape:  (2785, 27)


In [38]:
encoder_text[0]

'Wow!'

In [39]:
encoder_input_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1267],
      dtype=int32)

In [40]:
decoder_text[0]

'<start> वाह! <end>'

In [41]:
decoder_input_data[0]

array([  1, 752,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

#### Integer to Word converter for Decoder data

In [0]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [43]:
int_to_word_decoder[3]

'है।'

### Building Decoder Output

In [44]:
decoder_input_data.shape

(2785, 27)

In [0]:
import numpy as np

#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

In [49]:
decoder_input_data

array([[   1,  752,    2, ...,    0,    0,    0],
       [   1, 1446,    2, ...,    0,    0,    0],
       [   1, 1447,    2, ...,    0,    0,    0],
       ...,
       [   1, 2967,  750, ...,    0,    0,    0],
       [   1,  135,   60, ...,    0,    0,    0],
       [   1,  131,    6, ...,   26,  153,    2]], dtype=int32)

In [48]:
decoder_target_data

array([[7.520e+02, 2.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.446e+03, 2.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.447e+03, 2.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.967e+03, 7.500e+02, 1.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.350e+02, 6.000e+01, 1.333e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.310e+02, 6.000e+00, 5.110e+02, ..., 1.530e+02, 2.000e+00,
        0.000e+00]])

In [0]:
#decoder_t.word_index

In [0]:
#<start> yeh kitab hai <end>
#Yeh kitab hai <end> 0

In [0]:
decoder_input_data[0]

array([  1, 752,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

In [0]:
decoder_target_data[0]

array([752.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

#### Convert target data in one hot vector

In [0]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_t.word_index)+1)) #Vocab size + 1

In [51]:
decoder_target_one_hot.shape

(2785, 27, 2975)

In [0]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_t.word_index)+1)    

In [55]:
decoder_target_one_hot.shape

(2785, 27, 2975)

### Building the Training Model

In [0]:
#Define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

#### Build Encoder

In [0]:
#Input Layer
encoder_inputs = tf.keras.layers.Input(shape=(None,))

#Embedding layer
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, encoder_embedding_size)

#Get embedding layer output by feeding inputs
encoder_embedding_output = encoder_embedding(encoder_inputs)

#LSTM Layer and its output
x, state_h, state_c = tf.keras.layers.LSTM(rnn_units,return_state=True)(encoder_embedding_output)

#Build a list to feed Decoder - Sentence Embedding
encoder_states = [state_h, state_c]

In [60]:
encoder_embedding_output

<tf.Tensor 'embedding_2/embedding_lookup/Identity_1:0' shape=(?, ?, 50) dtype=float32>

#### Build Decoder

In [88]:
#Decode input - padded Target sentences
decoder_inputs = tf.keras.layers.Input(shape=(None,))

#Decoder Embedding layer
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1, decoder_embedding_size)

#Embedding layer output
decoder_embedding_output = decoder_embedding(decoder_inputs)

#Decoder RNN
decoder_rnn = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)

#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
x,_,_ = decoder_rnn(decoder_embedding_output, initial_state=encoder_states)

#Output Layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#Output of Dense layer
decoder_outputs = decoder_dense(x)
print(x.shape)

(?, ?, 256)


In [89]:
x.shape

TensorShape([Dimension(None), Dimension(None), Dimension(256)])

In [90]:
decoder_outputs

<tf.Tensor 'dense_4/truediv:0' shape=(?, ?, 2975) dtype=float32>

### Build Model using both Encoder and Decoder

In [0]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [92]:
model.output

<tf.Tensor 'dense_4/truediv:0' shape=(?, ?, 2975) dtype=float32>

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

### Train the model

In [94]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=64,
          epochs=2,
          validation_split=0.2)

Train on 2228 samples, validate on 557 samples
Epoch 1/2
2228/2228 [==============================] - 29s 13ms/sample - loss: 3.4357 - val_loss: 3.0096
Epoch 2/2
2228/2228 [==============================] - 26s 12ms/sample - loss: 1.6193 - val_loss: 3.0071


### Save the model for later reuse

In [0]:
model.save('seq2seq_training_translation.hd5')

# Building Model for Prediction

### Build the Encoder Model to predict Encoder States

In [0]:
encoder_model = tf.keras.models.Model(encoder_inputs, #Padded input sequences
                                      encoder_states) #Hidden state and Cell state at last time step

### Build the Decoder Model 
<p/>

<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li>Get Decoder Dense layer output</li>
        <li>Build Model</li></ol>

##### Step 1 - Define Input for both 'h' state and 'c' state initialization

In [0]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(rnn_units,))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(rnn_units,))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

##### Step 2 - Get Decoder RNN outputs along with h and c state

In [0]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs)

#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

#Why do we need this?
decoder_states = [state_h, state_c]

##### Step 3 - Get Decoder Dense layer output

In [0]:
decoder_outputs = decoder_dense(rnn_outputs)

##### Step 4 - Build Decoder Model

In [0]:
decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,  #Model inputs
                                      [decoder_outputs] + decoder_states)

# Predicting output from Seq2Seq model

##### Build a prediction function

In [0]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values - Sentence embedding
    decoder_initial_states_value = encoder_model.predict(input_seq)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['start']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    num_of_predictions = 0
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted word index with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter index
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == 'end' or num_of_predictions > max_decoder_seq_length):
            
            stop_loop = True
            continue
        
        #num_of_predictions += 1
        #predicted_sentence = predicted_sentence + ' ' + predicted_word
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

##### Call Prediction function on a random sentence

In [85]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)
start_num

161

In [86]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 5):
    input_seq = encoder_input_data[i : i+1]
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

--------
Input sentence:  Workers must have their hair cut short.
Predicted sentence:  मैं मैं
--------
Input sentence:  Would you mind turning down the volume?
Predicted sentence:  मैं मैं
--------
Input sentence:  You must observe the rules of the club.
Predicted sentence:  मैं मैं
--------
Input sentence:  You've got a nerve to say such a thing!
Predicted sentence:  मैं मैं
--------
Input sentence:  Your examination results are excellent.
Predicted sentence:  मैं मैं


##### Save encoder and decoder model

In [0]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('seq2seq_decoder_eng_hin.hd5')  #Decoder model

##### Save encoder and decoder tokenizers

In [0]:
import pickle

pickle.dump(encoder_t,open('encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('decoder_tokenizer_hin','wb'))

In [0]:
tf.keras.backend.clear_session()
edModelcheckpoint_path = "/home/narayana/wd/codes/finance/savedModels/edModel.ckpt"

nStepsIn = 1
nStepsOut = 1

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, activation='relu', input_shape=(nStepsIn, X_train.shape[2])))
model.add(tf.keras.layers.RepeatVector(nStepsOut))
model.add(tf.keras.layers.LSTM(128, activation='relu', return_sequences=True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_classes, activation='softmax')))
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=edModelcheckpoint_path,
                                                 save_weights_only=True,
                                                 monitor='loss',
                                                 mode = 'min',
                                                 save_best_only=True,
                                                 verbose=1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train_copy = y_train.copy()
y_test_copy = y_test.copy()
y_train_copy = np.reshape(y_train_copy, (y_train_copy.shape[0], 1, y_train_copy.shape[1]))
y_test_copy = np.reshape(y_test_copy, (y_test_copy.shape[0], 1, y_test_copy.shape[1]))

model.fit(X_train, y_train_copy, epochs=100, validation_data=(X_test, y_test_copy), batch_size=32, callbacks=[cp_callback])
# Loads the weights
model.load_weights(edModelcheckpoint_path)
loss_and_metrics = model.evaluate(X_train, y_train_copy)
print(loss_and_metrics)
loss_and_metrics = model.evaluate(X_test, y_test_copy)
print(loss_and_metrics)